In [1]:
import numpy as np
import pandas as pd
import mysql.connector as sql
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim as nm
import folium as fm
from folium.plugins import MarkerCluster
from tqdm.notebook import tqdm

#%matplotlib qt


In [2]:
#IMPORTING ALL THREE DATA FRAMES 
df1 = pd.read_csv('events_data.csv')
server = sql.connect(host='cpanel.insaid.co', database='Capstone1', user='student', password='student')

df2 = pd.read_sql('SELECT * FROM gender_age_train', con=server)
df3 = pd.read_sql('SELECT * FROM phone_brand_device_model', con=server)
server.close

#ADDITIONAL REQUIRED INFO
eng_names = np.array([['华为','Huawei'],
                      ['小米','Xiaomi'],
                      ['三星','Samsung'],
                      ['vivo','vivo'],
                      ['OPPO','OPPO'],
                      ['魅族','Meizu'],
                      ['酷派','Coolpad'],
                      ['乐视','LeEco'],
                      ['联想','Lenovo'],
                      ['HTC','HTC']])

In [3]:
# Some output formatting assistance
def line_break():
    print('-------------------------------------------------------------------------------------')



# Check 1 - Whether longitude and latitude are always missing in pairs:
# This code creates two dataframes, containing the index values (row numbers) of the rows where either one is missing, and then checks if the two dataframes are identical.
# If they are identical, then they are always missing in pairs.
Check1 = df1[df1['latitude'].isnull()].index.equals(df1['longitude'].isnull().index)
print("Check 1 - Latitude and Longitude are always missing in pairs - " + str(Check1))

line_break()

# Check 2 - Whether each city only exists in one state
# This code creates a dataframe containing unique city-state pairs, and the length of this dataframe is compared to the number of unique cities.
# If they are of different length, then we cannot calculate the state using the city
city_state_pairs = df1[df1['state'].notnull()][['city','state']].drop_duplicates()
unique_cities = df1['city'].nunique()
print('Check 2 - There are ' + str(len(city_state_pairs)) + ' unique city-state pairs, and ' + str(unique_cities) + ' unique cities')
#These two numbers are not equal. This means that some cities are present in multiple states. To verify:
city_state_pairs['city'].value_counts()[0:11]

line_break()

# Check 3 - Whether we can use coordinates to find the state
Check3a = df1[df1['latitude'].isnull()].index.tolist()
Check3b = df1[df1['state'].isnull()].index.tolist()
for i in Check3a:
    if i in Check3b:
        print('Check 3 - In row ' + str(i) + ', both the coordinates and the city are missing')
# This prints nothing; therefore, we can use coordinates to find the city.
# This is foolproof, as each coordinate pair MUST refer to only one state

line_break()

# Check 4 - Keep track of number of unique entries in "state"
# After adding the states using geopy, if the syntax is wrong, then this will help detect that:
print('Check 4 - There are ' + str(df1['state'].nunique()) + ' unique entries in the "state" column.')



Check 1 - Latitude and Longitude are always missing in pairs - False
-------------------------------------------------------------------------------------
Check 2 - There are 943 unique city-state pairs, and 933 unique cities
-------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------
Check 4 - There are 32 unique entries in the "state" column.


In [4]:
# This is a list containing the index values of all rows where state is a null value.
null_records = df1[df1['state'].isnull()].index.tolist()
# This section of code fills in those values via geopy and a loop over the rows with null values
geolocator = nm(user_agent="my-applications")
for i in null_records:
    location = geolocator.reverse([df1.at[i,'latitude'],df1.at[i,'longitude']])
    # the code for location is taken from https://www.geeksforgeeks.org/python-remove-spaces-from-a-string/
    df1.at[i,'state'] = location.raw['address'].get('state', '').replace(' ','')

print(df1['state'].nunique())

## THIS DOES THE FILTERING FOR MAHARASHTRA
#df1 = df1[df1['state'] == "Maharashtra"]

32


In [6]:
df1.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3252950 entries, 0 to 3252949
Data columns (total 7 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   event_id   3252950 non-null  int64  
 1   device_id  3252497 non-null  float64
 2   timestamp  3252950 non-null  object 
 3   longitude  3252527 non-null  float64
 4   latitude   3252527 non-null  float64
 5   city       3252950 non-null  object 
 6   state      3252950 non-null  object 
dtypes: float64(3), int64(1), object(3)
memory usage: 173.7+ MB


In [15]:
# Studying the null values in the coordinates. 
# This dataframe contains only the rows with null values in the coordinates
t1 = df1[df1['longitude'].isnull()]
print(t1.info())
line_break()
# This shows the device_ids present in the records with missing coordinates. 
# For MHA, there are 3 missing ids with 21 missing coordinates each. For all-India, there are 21 missing ids with 14-22 missing coordinates each.
print(t1['device_id'].value_counts())
# This stores the unique device_ids, so we can study them across the whole database.
bad_coords_id = t1['device_id'].value_counts().index

<class 'pandas.core.frame.DataFrame'>
Int64Index: 423 entries, 8015 to 3249260
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   event_id   423 non-null    int64  
 1   device_id  423 non-null    float64
 2   timestamp  423 non-null    object 
 3   longitude  0 non-null      float64
 4   latitude   0 non-null      float64
 5   city       423 non-null    object 
 6   state      423 non-null    object 
dtypes: float64(3), int64(1), object(3)
memory usage: 26.4+ KB
None
-------------------------------------------------------------------------------------
 3.058488e+17    22
 4.580179e+18    22
 4.375040e+18    22
 1.779631e+18    21
-8.790560e+18    21
 5.115185e+18    21
 7.018600e+18    21
 3.738280e+18    21
 5.731369e+18    21
 4.782582e+18    21
 2.350979e+18    21
 7.662139e+18    21
-6.033798e+18    21
 5.518660e+18    21
 5.375599e+18    21
-7.364841e+18    21
 7.983871e+18    21
 4.865607e+18    21
 1.320051e+18  

In [17]:
# These count the number of unique coordinate pairs that the device_ids have across the whole database
# Note that each of the device_ids has one unique coordinate pair associated with it across ALL records. This means that the device was used from the same location all the time
# There are multiple hundred rows for each device_id with the coordinates present, so we can safely substitute the missing values with the device_id's coordinates.
for i in range(0,len(bad_coords_id)):
    print(df1[df1['device_id'] == bad_coords_id[i]][['latitude','longitude']].value_counts())
    print(df1[df1['device_id'] == bad_coords_id[i]][['latitude','longitude']].info())
    line_break()

latitude   longitude
26.987566  75.865534    560
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 582 entries, 1497 to 3240541
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   latitude   560 non-null    float64
 1   longitude  560 non-null    float64
dtypes: float64(2)
memory usage: 13.6 KB
None
-------------------------------------------------------------------------------------
latitude  longitude
26.96022  75.835756    599
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 621 entries, 813 to 3251871
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   latitude   599 non-null    float64
 1   longitude  599 non-null    float64
dtypes: float64(2)
memory usage: 14.6 KB
None
-------------------------------------------------------------------------------------
latitude   longitude
26.980345  75.824489    577
dtype: int64
<class 'pan

In [18]:
# Doing the substitution here
for i in range(0,len(bad_coords_id)):
    values = {}
    replacement = df1[df1['device_id'] == bad_coords_id[i]][['latitude','longitude']].value_counts().index[0]
    values['latitude'] = replacement[0]
    values['longitude'] = replacement[1]
    df1[df1['device_id'] == bad_coords_id[i]] = df1.fillna(value = values)
df1.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3252950 entries, 0 to 3252949
Data columns (total 7 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   event_id   3252950 non-null  int64  
 1   device_id  3252497 non-null  float64
 2   timestamp  3252950 non-null  object 
 3   longitude  3252950 non-null  float64
 4   latitude   3252950 non-null  float64
 5   city       3252950 non-null  object 
 6   state      3252950 non-null  object 
dtypes: float64(3), int64(1), object(3)
memory usage: 173.7+ MB


In [19]:
# THIS BIT IS COMMENTED OUT FOR NOW, PENDING REVIEW FOR WHETHER THIS WORKS FOR ALL RECORDS RATHER THAN JUST MAHARASHTRA

# This takes out the 9 points outside of Maharashtra, based on folium visualization (removed that code, as it takes AGES). 
# The most eastern point of the 9 was in Kabul at ~69 longitude, so a cutoff of 72 is used here.

# df1 = df1[df1['longitude'] > 72]
# df1.info(show_counts=True)

In [ ]:
# Everything past this is under review.

#Let us assume that the coordinates and device_ids are one-to-one.

In [36]:
# a1 is full of unique triplets
a1 = df1[['device_id','longitude','latitude']].drop_duplicates()
a1.info()
# conclusion - 21 missing device_ids
df1['device_id'].drop_duplicates.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60907 entries, 0 to 3252798
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   device_id  60886 non-null  float64
 1   longitude  60907 non-null  float64
 2   latitude   60907 non-null  float64
dtypes: float64(3)
memory usage: 1.9 MB


453

In [23]:
# Check 5 - Whether each device_id has got a unique corresponding coordinate.
# Same number of coordinate entires after dropping duplicates even if we include device_id
# This means that the above check is True
print(df1.info(show_counts = True))
print(df1[['device_id','longitude','latitude']].drop_duplicates().info())
print(df1[['device_id']].drop_duplicates().info())
print(df1[['longitude','latitude']].drop_duplicates().info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3252950 entries, 0 to 3252949
Data columns (total 7 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   event_id   3252950 non-null  int64  
 1   device_id  3252497 non-null  float64
 2   timestamp  3252950 non-null  object 
 3   longitude  3252950 non-null  float64
 4   latitude   3252950 non-null  float64
 5   city       3252950 non-null  object 
 6   state      3252950 non-null  object 
dtypes: float64(3), int64(1), object(3)
memory usage: 173.7+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 60907 entries, 0 to 3252798
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   device_id  60886 non-null  float64
 1   longitude  60907 non-null  float64
 2   latitude   60907 non-null  float64
dtypes: float64(3)
memory usage: 1.9 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 60866 entries, 0 to 3252798
Data col

In [33]:
df1[['device_id','longitude','latitude']].drop_duplicates()['device_id'].nunique()
df1[df1['device_id'].notnull()][['longitude','latitude']].drop_duplicates()

,longitude,latitude
0,77.225676,28.730140
1,88.388361,22.660325
2,77.256809,28.757906
3,80.343613,13.153332
4,85.997745,23.842609
...,...,...
3251857,80.290241,13.150984
3252013,72.886717,19.049232
3252525,77.633307,13.065801
3252794,76.422242,26.947862


In [11]:
# Studying the null values in the device_ids. Yes, this is literally a duplicate of what was done for the coordinates.
# This dataframe contains only the rows with null values in device_id
t2 = df1[df1['device_id'].isnull()]
print(t2.info())
line_break()
# This shows the coordinate pairs that are present with missing device_ids. 
# Note that there are only 3 unique pairs, with 24 missing device_ids each.
print(t2[['longitude','latitude']].value_counts())
# This stores the 3 unique coordinate, so we can study them across the whole database.
bad_device_id = t2[['longitude','latitude']].value_counts().index

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72 entries, 45052 to 3135318
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   event_id   72 non-null     int64  
 1   device_id  0 non-null      float64
 2   timestamp  72 non-null     object 
 3   longitude  72 non-null     float64
 4   latitude   72 non-null     float64
 5   city       72 non-null     object 
 6   state      72 non-null     object 
dtypes: float64(3), int64(1), object(3)
memory usage: 4.5+ KB
None
-------------------------------------------------------------------------------------
longitude  latitude 
73.860165  18.566925    24
73.862756  18.628057    24
73.926499  18.614812    24
dtype: int64


In [12]:
# These take the three aforementioned coordinate pairs and counts the device_id for each pair in the full database
# Note that for each pair, only one device_id turns up
# This means that for the 24 records missing device_ids, we can assume that the same device_id is used as for the hundreds of records at that coordinate which have device_id present.
print(df1.loc[(df1['longitude'] == bad_device_id[0][0]) & (df1['latitude'] == bad_device_id[0][1])]['device_id'].value_counts())
line_break()
print(df1.loc[(df1['longitude'] == bad_device_id[1][0]) & (df1['latitude'] == bad_device_id[1][1])][['device_id']].value_counts())
line_break()
print(df1.loc[(df1['longitude'] == bad_device_id[2][0]) & (df1['latitude'] == bad_device_id[2][1])][['device_id']].value_counts())

8.539871e+18    1793
Name: device_id, dtype: int64
-------------------------------------------------------------------------------------
device_id   
9.027086e+18    1767
dtype: int64
-------------------------------------------------------------------------------------
device_id   
1.771586e+17    1698
dtype: int64


In [13]:
# Doing the substitution here
for i in range(0,3):
    replacement = df1.loc[(df1['longitude'] == bad_device_id[i][0]) & (df1['latitude'] == bad_device_id[i][1])]['device_id'].value_counts().index[0]
    (df1.loc[(df1['longitude'] == bad_device_id[i][0]) & (df1['latitude'] == bad_device_id[i][1])]) = df1.fillna(value = replacement)
df1.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 677159 entries, 9 to 3252946
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   event_id   677159 non-null  int64  
 1   device_id  677159 non-null  float64
 2   timestamp  677159 non-null  object 
 3   longitude  677159 non-null  float64
 4   latitude   677159 non-null  float64
 5   city       677159 non-null  object 
 6   state      677159 non-null  object 
dtypes: float64(3), int64(1), object(3)
memory usage: 57.5+ MB


In [16]:
df1['device_id'] = df1['device_id'].astype('int64')

In [52]:
df4 = pd.merge(left=df1,right=df2,how='left',on='device_id')

In [53]:
df4.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 677159 entries, 0 to 677158
Data columns (total 10 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   event_id   677159 non-null  int64  
 1   device_id  677159 non-null  int64  
 2   timestamp  677159 non-null  object 
 3   longitude  677159 non-null  float64
 4   latitude   677159 non-null  float64
 5   city       677159 non-null  object 
 6   state      677159 non-null  object 
 7   gender     3177 non-null    object 
 8   age        3177 non-null    float64
 9   group      3177 non-null    object 
dtypes: float64(3), int64(2), object(5)
memory usage: 56.8+ MB


In [47]:
w1 = np.zeros([74645,2],dtype='int64')
w1[:,0] = np.array(df2['device_id'].tolist())

In [48]:
for i in tqdm(range(0,len(w1[:,0]))):
    w1[i,1] = (df1['device_id'] == w1[i,0]).sum() 

  0%|          | 0/74645 [00:00<?, ?it/s]

In [49]:
w1[:,1].sum()

3177

In [44]:
(df2['device_id'].tolist())

[-8076087639492063270,
 -2897161552818060146,
 -8260683887967679142,
 -4938849341048082022,
 245133531816851882,
 -1297074871525174196,
 236877999787307864,
 -8098239495777311881,
 176515041953473526,
 1596610250680140042,
 9032155484127182494,
 7477216237379271436,
 2478205222798310601,
 6352067998666467520,
 -7605360767281960867,
 1508636020748379883,
 4380872794486415327,
 -1039701474753771322,
 -6876541075223249434,
 6287938418661076759,
 -6029676723224952628,
 8026504930081700361,
 1118970699362079126,
 -7271319853104672050,
 -9216966316587614491,
 -4913644674161904021,
 2313145512701915151,
 4309468945717577632,
 -4547337748907986143,
 -7216269352973401877,
 -7303294878482532208,
 7630241409816671260,
 7106922851105968241,
 -5825546292084395293,
 5886404762741543631,
 3444770318780911055,
 3549658489155683094,
 -7360652256644378412,
 -3207654344601946580,
 -1279883160764317574,
 3014072209520807550,
 -2871140782885206484,
 -5070177130701218827,
 -1539979963116391306,
 -5349338524